In [8]:
# Imports
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By

In [62]:

from time import time
# Call selenium webdriver
driver = webdriver.Chrome()
# Fill login page with user and password
base_url = "https://devopsdays.org/events"
driver.get(base_url)

all_elements = driver.find_elements(By.CSS_SELECTOR, "body > div > div > div")
print(len(all_elements))

all_elements = all_elements[0].find_elements(By.CLASS_NAME, "row")
print(len(all_elements))

upcoming_events = all_elements[1]
past_events = all_elements[3]

1


In [95]:
months = ["JANUARY", "FEBRUARY", "MARCH", "APRIL", "MAY", "JUNE",
          "JULY", "AUGUST", "SEPTEMBER", "OCTOBER", "NOVEMBER", "DECEMBER", "TBD"]

In [143]:
raw_events_df = pd.DataFrame(columns=["month", "year", "str_date", "city"])

month_year = ""
for item in upcoming_events.text.split("\n"):
    if any(map(lambda x: x in item, months)):
        month_year = item
    else:
        tam = len(raw_events_df)
        if " " in month_year:
            month, year = month_year.split(" ")
            str_date, city = item.split(":")
        else:
            city = item
            month, year, str_date = month_year, month_year, month_year
        raw_events_df.loc[tam] = [month.strip(), year.strip(), str_date.strip(), city.strip()]

month_year = ""
for item in filter(lambda x: x != '', past_events.text.split("\n")):
    try:
        month_year = int(item)
    except:
        tam = len(raw_events_df)
        raw_events_df.loc[tam] = ['', month_year, '', item]
        
raw_events_df.head(5)

,month,year,str_date,city
0,MARCH,2024,Mar 13 - 14,Salt Lake City
1,MARCH,2024,Mar 15,Los Angeles
2,MARCH,2024,Mar 18 - 19,Kraków
3,MARCH,2024,Mar 21 - 22,Singapore
4,APRIL,2024,Apr 6,Goiânia


In [144]:
driver.close()

In [216]:
cities_df = pd.read_csv("events_city_country.csv")
cities_df.drop_duplicates(subset="city", inplace=True)

join_df = raw_events_df.join(cities_df.set_index("city"), on="city", how="left")
join_df = join_df[['month', 'year', 'city', 'country']]
# Fix missing
join_df.loc[456] = ["", "2022", "tbd", "Ukraine"]
join_df.loc[505] = ["", "2023", "tbd", "Ukraine"]
join_df.loc[507] = ["", "2023", "Chicago", "United States"]

seasons = {"\(Spring\)": "", "\(Summer\)": "", "\(Fall\)": "", "\(Winter\)": "","\(Winterthur\)": "", "\(Autumn\)": ""}
# Regex replace:  (Spring), (Summer), (Fall), (Winter), (Winterthur) by ""
join_df[['month', 'city']] = join_df[['month', 'city']].replace(seasons, regex=True)
join_df[['month', 'city', 'country']] = join_df[['month', 'city', 'country']].map(str.strip)

join_df.to_csv("events.csv", index=False)
